# Ai Mercado
 yo y los demas

##  Descripcion de variables

## Importacion de librerias y bases

### Autenticador del Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


### Librerias

In [2]:
!pip install pandas-profiling
!pip install pyxlsb
!pip install sweetviz

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.4/324.4 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 390.6/390.6 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.5/296.5 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 686.1/686.1 kB 31.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.8/104.8 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 79.3 MB/s eta 0:00:00
  Created wheel for htmlmin: filename=htmlmin-0.1.12-py3-none-any.whl size=27081 sha256=a67a5544abf6bfd28653e24fd0aef543eaee7eec1c2ce3810de11e00fd1f8d1b
  Stored in directory: /root/.cache/pip/wheels/dd/91/29/a79cecb328d01739e64017b6fb9a1ab9d8cb1853098ec5966d
Successfully built htmlmin
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.1/15.1 MB 73.4 MB/s eta 0:00:00


In [3]:
import os
import re
import math
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
from ydata_profiling import ProfileReport
import sweetviz as sv
from google.colab import files

### Funciones

In [4]:
def missing_values(data):
  df = pd.DataFrame()
  for col in list(data):
    unique_values = data[col].unique()
    try:
      unique_values = np.sort(unique_values)
    except:
      pass
    nans = round(pd.isna(data[col]).sum()/data.shape[0]*100, 1)
    # zeros = round((data[col] == 0).sum()/data.shape[0]*100, 1)
    df = pd.concat([df, pd.DataFrame([[col, len(unique_values), nans]], columns=['variable', 'unique value', 'NaN %'])], ignore_index=True)
  return df.rename(columns={'NaN %':'Nan %', 'Zeros %': 'Zeros %'}).sort_values('Nan %', ascending=False)


### Ruta de los datos

Ruta de las bases de datos desde el 2013 hasta septiembre de 2024

In [5]:
pd1 = '/content/drive/MyDrive/Proyecto IA/Base de datos final/datos_manizales_completados.csv'


In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Cargar bases de datos

Carga de cada una de las bases de datos con eliminacion de columnas vacias

In [10]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from collections import defaultdict

# Cargar los datos
pd1 = '/content/drive/MyDrive/Proyecto IA/Base de datos final/datos_manizales_completados.csv'
df1 = pd.read_csv(pd1, sep=",", encoding='latin1')
df = df1.loc[:, ~df1.columns.str.contains('^Unnamed')]

# Crear un defaultdict para almacenar precios de cada producto
productos_precios_dict = defaultdict(list)

# Recorrer el DataFrame y llenar el defaultdict
for _, row in df.iterrows():
    producto = row['Producto']
    precio = row['Precio_kg']
    productos_precios_dict[producto].append(precio) #falta ordenar por fecha todos los vecotres o al menos no esta especificado

# Ajustar las listas de precios para que todas tengan la misma longitud
max_length = max(len(precios) for precios in productos_precios_dict.values())
for producto, precios in productos_precios_dict.items():
    precios.extend([np.nan] * (max_length - len(precios)))  # Rellenar con NaN si es necesario

# Convertir el defaultdict a un DataFrame
productos_precios_df = pd.DataFrame(productos_precios_dict)

# Rellenar valores NaN con 0 o el valor deseado
productos_precios_df = productos_precios_df.fillna(0)

# Calcular la matriz de correlación entre los vectores de precios
correlation_matrix = productos_precios_df.corr()

# Graficar el mapa de calor de la matriz de correlación
plt.figure(figsize=(14, 12))
sns.heatmap(correlation_matrix, annot=False, cmap='coolwarm', cbar=True, xticklabels=productos_precios_df.columns, yticklabels=productos_precios_df.columns)
plt.title('Matriz de correlación de precios por producto')
plt.xticks(rotation=90)
plt.yticks(rotation=0)
plt.show()


NameError: name 'df1' is not defined

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from collections import defaultdict
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

# Cargar los datos
pd1 = '/content/drive/MyDrive/Proyecto IA/Base de datos final/datos_manizales_completados.csv'
df1 = pd.read_csv(pd1, sep=",", encoding='latin1')
df = df1.loc[:, ~df1.columns.str.contains('^Unnamed')]

# Asegurarse de que la columna de fecha esté en el formato correcto
df['Fecha'] = pd.to_datetime(df['Fecha'])  # Cambia 'Fecha' al nombre real de la columna de fecha

# Ordenar el DataFrame por fecha
df = df.sort_values(by='Fecha')

# Crear un defaultdict para almacenar precios de cada producto
productos_precios_dict = defaultdict(list)

# Recorrer el DataFrame y llenar el defaultdict
for _, row in df.iterrows():
    producto = row['Producto']
    precio = row['Precio_kg']
    productos_precios_dict[producto].append(precio)

# Ajustar las listas de precios para que todas tengan la misma longitud
max_length = max(len(precios) for precios in productos_precios_dict.values())
for producto, precios in productos_precios_dict.items():
    precios.extend([np.nan] * (max_length - len(precios)))  # Rellenar con NaN si es necesario

# Convertir el defaultdict a un DataFrame
productos_precios_df = pd.DataFrame(productos_precios_dict)

# Rellenar valores NaN con 0 o el valor deseado
productos_precios_df = productos_precios_df.fillna(0)

# Calcular la matriz de correlación entre los vectores de precios
correlation_matrix = productos_precios_df.corr()

# Graficar el mapa de calor de la matriz de correlación
plt.figure(figsize=(14, 12))
sns.heatmap(correlation_matrix, annot=False, cmap='coolwarm', cbar=True, xticklabels=productos_precios_df.columns, yticklabels=productos_precios_df.columns)
plt.title('Matriz de correlación de precios por producto')
plt.xticks(rotation=90)
plt.yticks(rotation=0)
plt.show()

# Definir el producto a predecir
producto_a = 'Producto_A'  # Cambia esto al nombre real del producto

# Establecer un umbral de correlación
umbral = 0.5  # Ajusta este valor según sea necesario
productos_relacionados = correlation_matrix[producto_a][correlation_matrix[producto_a].abs() > umbral].index.tolist()

# Filtrar los datos históricos para las características y la variable objetivo
X_historico = productos_precios_df[productos_relacionados].drop(columns=[producto_a])
y_historico = productos_precios_df[producto_a]

# Dividir los datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_historico, y_historico, test_size=0.2, random_state=42)

# Crear y entrenar el modelo
model = LinearRegression()
model.fit(X_train, y_train)

# Crear un DataFrame con los precios futuros estimados
nuevos_datos_futuros = pd.DataFrame({
    'Producto_B': [valor_futuro_B],  # Cambia estos valores según tus estimaciones
    'Producto_C': [valor_futuro_C],
    # Agrega otros productos según sea necesario
})

# Hacer predicciones
precio_predicho = model.predict(nuevos_datos_futuros)

# Mostrar el precio predicho
print(f'El precio predicho de {producto_a} es: {precio_predicho[0]:.2f}')
